In [ ]:
import os
import cv2
import torch
import numpy as np
from facenet_pytorch import MTCNN

# Qurilmani aniqlash
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Modelni yuklash
mtcnn = MTCNN(keep_all=True, device=device)

# Dataset papkasini yaratish
DATASET_PATH = "dataset"
if not os.path.exists(DATASET_PATH):
    os.makedirs(DATASET_PATH)

# Yangi ID yaratish funksiyasi
def get_new_id():
    existing_ids = [int(folder) for folder in os.listdir(DATASET_PATH) if folder.isdigit()]
    return max(existing_ids) + 1 if existing_ids else 1

# Kamera ochish
cap = cv2.VideoCapture(0)

# Foydalanuvchi nomini kiritish
person_name = input("Foydalanuvchi ismini kiriting: ").strip()
person_id = get_new_id()  # Yangi ID yaratish
save_path = os.path.join(DATASET_PATH, str(person_id))

# Rasm saqlash uchun papka yaratish
os.makedirs(save_path, exist_ok=True)

print(f"{person_name} uchun ID: {person_id}")
print(f"Rasmlar {save_path} ichiga saqlanadi.")

frame_count = 0
max_frames = 50  # Har bir shaxs uchun 50 ta rasm

while frame_count < max_frames:
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    boxes, _ = mtcnn.detect(rgb_frame)

    if boxes is not None and len(boxes) > 0:
        # Eng katta yuzni tanlash (eng yaqin odam)
        biggest_box = max(boxes, key=lambda b: (b[2] - b[0]) * (b[3] - b[1]))
        x1, y1, x2, y2 = [int(b) for b in biggest_box]
        face = rgb_frame[y1:y2, x1:x2]

        if face.size > 0:
            frame_count += 1
            file_name = f"{frame_count}.jpg"
            file_path = os.path.join(save_path, file_name)

            # Rasmni saqlash
            cv2.imwrite(file_path, cv2.cvtColor(face, cv2.COLOR_RGB2BGR))

            # Yuzni belgilash
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"Saving {frame_count}/{max_frames}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv2.imshow("Face Capture", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

print(f"{person_name} uchun 50 ta rasm saqlandi. ID: {person_id}")

cap.release()
cv2.destroyAllWindows()


In [13]:
import os
import cv2
import torch
import numpy as np
from facenet_pytorch import MTCNN, InceptionResnetV1
from scipy.spatial.distance import cosine

# Qurilmani aniqlash
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Modelni yuklash
mtcnn = MTCNN(keep_all=False, device=device)  # Eng yaqin yuzni olish
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Dataset papkasi
DATASET_PATH = "dataset"

# Gamma correction (yorug‘likni yaxshilash)
def adjust_gamma(image, gamma=1.5):
    inv_gamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** inv_gamma * 255 for i in range(256)]).astype("uint8")
    return cv2.LUT(image, table)

# Embeddings yuklash
def load_embeddings():
    embeddings = {}
    for person_id in os.listdir(DATASET_PATH):
        person_folder = os.path.join(DATASET_PATH, person_id)
        if not os.path.isdir(person_folder):
            continue
        for file in os.listdir(person_folder):
            img_path = os.path.join(person_folder, file)
            img = cv2.imread(img_path)
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            face = mtcnn(img_rgb)
            if face is not None:
                with torch.no_grad():
                    emb = resnet(face.unsqueeze(0).to(device)).cpu().numpy()
                    embeddings[person_id] = emb
    return embeddings

# Ma'lumotlarni yuklash
database = load_embeddings()

# Kamera ochish
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = adjust_gamma(frame)  # Yorug‘likni yaxshilash
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face = mtcnn(rgb_frame)

    if face is not None:
        with torch.no_grad():
            query_emb = resnet(face.unsqueeze(0).to(device)).cpu().numpy()

        best_match = None
        best_score = float("inf")

        # Eng yaqin odamni topish
    for person_id, emb in database.items():
        score = cosine(query_emb.flatten(), emb.flatten())  
        if score < best_score:
            best_score = score
            best_match = person_id

        if best_match and best_score < 0.5:  # Tanib olish uchun threshold
            name = f"ID: {best_match}"
        else:
            name = "Not Recognized"

        cv2.putText(frame, name, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


